# 한영 번역 서비스 만들기 M2M100

In [1]:
# !pip install sentencepiece

In [3]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

ko_text = "이것은 한국어/영어 번역기입니다. m2m100 모델로 만들었어요. 참 좋네요"
eng_text = "This is a Korean-English translator. It's an m2m100 model. That's great"

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")


tokenizer.src_lang = "ko"
encoded_ko = tokenizer(ko_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_ko, forced_bos_token_id=tokenizer.get_lang_id("en"))
translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
print(translated_text)


tokenizer.src_lang = "en"
encoded_en = tokenizer(eng_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.get_lang_id("ko"))
translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
print(translated_text)

This is a Korean/English translator. made in m2m100 model. great
이것은 한국어 - 영어 번역기입니다.이것은 m2m100 모델입니다.


GPT가 짜준 코드

In [6]:
import gradio as gr
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Load the model and tokenizer
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

def translate_text(input_text, source_lang):
    if source_lang == "Korean (ko)":
        tokenizer.src_lang = "ko"
        encoded = tokenizer(input_text, return_tensors="pt")
        generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("en"))
        translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    else:
        tokenizer.src_lang = "en"
        encoded = tokenizer(input_text, return_tensors="pt")
        generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.get_lang_id("ko"))
        translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    
    return translated_text

with gr.Blocks() as app:
    gr.Markdown("# Korean-English Translation Service")
    
    with gr.Row():
        source_lang = gr.Radio(["Korean (ko)", "English (en)"], label="Select Source Language")
        input_text = gr.Textbox(label="Input Text", placeholder="Enter text to translate...")
        output_text = gr.Textbox(label="Translated Text", interactive=False)
        translate_btn = gr.Button("Translate")
        
        translate_btn.click(
            fn=translate_text,
            inputs=[input_text, source_lang],
            outputs=output_text
        )
    
    gr.Markdown("### Example Inputs:")
    gr.Examples(
        examples=[
            ["이것은 번역기입니다.", "Korean (ko)"],
            ["This is a translator.", "English (en)"]
        ],
        inputs=[input_text, source_lang],
        outputs=output_text,
        fn=translate_text
    )

app.launch()

/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
